# Оценки двух долей

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.
<br><br>
За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо.
<br><br>
Необходимо оценить, насколько вероятность инфаркта снижается при приёме аспирина.

In [69]:
import numpy as np
import scipy
from scipy import stats

In [70]:
n_aspirin=11037
n_placebo=11034
p_aspirin=104/n_aspirin
p_placebo=189/n_placebo

In [55]:
print("Точечная оценка вероятности возникновения инфаркта при приеме аспирина: %.4f"%p_aspirin)
print("Точечная оценка вероятности возникновения инфаркта при приеме плацебо: %.4f"%p_placebo)

Точечная оценка вероятности возникновения инфаркта при приеме аспирина: 0.0094
Точечная оценка вероятности возникновения инфаркта при приеме плацебо: 0.0171


## Разность двух долей

In [84]:
print('При использовании точечных оценок вероятностей получается,что риск возникновения \
инфаркта при приеме аспирина снижается на:\n%.4f*100 процента.'%(p_placebo-p_aspirin))

При использовании точечных оценок вероятностей получается,что риск возникновения инфаркта при приеме аспирина снижается на:
0.0077*100 процента.


Так как точечные оценки не всегда точны, построим интервальную оценку разности полученных вероятностей.

Выборки независимые, поэтому будем использовать следующие формулы:<br>
  
|<div style="width:50px"></div> |<div style="width:100px">$X_1$</div>|<div style="width:100px">$X_2$</div>| 
|:-:|:-:|:-:|
|1|a|b|
|0|c|d|
|$\sum$ | $n_1$| $n_2$ |
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [72]:
def proportions_diff_confint(p1,p2,n1,n2,alpha):
    z=stats.norm.ppf(1-alpha/2)
    upper_lim=p1-p2+z*(np.sqrt(p1*(1-p1)/n1+p2*(1-p2)/n2))
    lower_lim=p1-p2-z*(np.sqrt(p1*(1-p1)/n1+p2*(1-p2)/n2))
    return (lower_lim, upper_lim)

In [78]:
print('95-процентный доверительный интервал для разности вероятностей возникновения \
инфарктов при приеме аспирина и плацебо: \n[%.4f, %.4f]'%proportions_diff_confint(p_placebo,p_aspirin,n_placebo,n_aspirin,0.05))

95-процентный доверительный интервал для разности вероятностей возникновения инфарктов при приеме аспирина и плацебо: 
[0.0047, 0.0107]


## Отношение шансов

Теперь рассчитаем шансы инфаркта в каждой из контрольных групп. Для бернуллиевских случайных величин $X\sim Ber(p)$ их вычисляют как $\frac{p}{1-p}$. Соответственно, для оценки шансов по выборкам нужно вместо $p$ подставить $\hat{p}$.

In [86]:
chance_aspirin=p_aspirin/(1-p_aspirin)
chance_placebo=p_placebo/(1-p_placebo)
print('Шанс возникновения инфаркта при приеме аспирина снижается в %.4f раз.'%(chance_placebo/chance_aspirin))

Шанс возникновения инфаркта при приеме аспирина снижается в 1.8321 раз


Теперь оценим отношение шансов интервально. Снова будем находить 95% доверительный интервал. Для этого будем использовать бутстрапированные выборки.

In [102]:
effect_aspirin=np.hstack((np.repeat(1,104),np.repeat(0,n_aspirin-104)))
effect_placebo=np.hstack((np.repeat(1,189),np.repeat(0,n_placebo-189)))

In [104]:
def bootstrap_samples(data, n_samples):
    indices=np.random.randint(0,len(data),(n_samples, len(data)))
    samples=data[indices]
    return samples

In [144]:
np.random.seed(0)
chances_aspirin=list(map(lambda x:x.mean()/(1-x.mean()),bootstrap_samples(effect_aspirin,1000)))
chances_placebo=list(map(lambda x:x.mean()/(1-x.mean()),bootstrap_samples(effect_placebo,1000)))

In [145]:
len(chances_placebo)

1000

In [146]:
def chance_confint(data1, data2, alpha):
    data=[data1[i]/data2[i] for i in range(len(data1))]
    limits=np.percentile(data, [100*alpha/2, 100*(1-alpha/2)])
    return (limits[0],limits[1])

In [147]:
print('95-процентный доверительный интервал для отношения шанса возникновения \
инфаркта при приеме плацебо к шансу \
при приеме аспирина: \n[%.4f, %.4f]'%chance_confint(chances_placebo,chances_aspirin,0.05))

95-процентный доверительный интервал для отношения шанса возникновения инфаркта при приеме плацебо к шансу при приеме аспирина: 
[1.4442, 2.3432]
